In [ ]:
pip install numpy matplotlib imageio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

# Funcion que genera matriz de traslacion dados desplazamientos tx y ty
def traslacion_matriz(tx, ty):
    return np.array([
        [1, 0, tx],  # no se modifica eje x ni y, solo se añade desplazamiento en x
        [0, 1, ty],  # no se modifica eje y, se añade desplazamiento en y
        [0, 0, 1]
    ])

# Funcion que genera matriz de rotacion para un angulo en grados
def rotacion_matriz(angulo_grados):
    angulo_rad = np.radians(angulo_grados)  # convertir grados a radianes
    cos_theta = np.cos(angulo_rad)
    sin_theta = np.sin(angulo_rad)
    return np.array([
        [cos_theta, -sin_theta, 0],  # rotacion en el plano XY
        [sin_theta,  cos_theta, 0],  # rotacion en el plano XY
        [0,          0,         1]
    ])

# Funcion que genera matriz de escala dados factores sx y sy
def escala_matriz(sx, sy):
    return np.array([
        [sx, 0,  0],  # escala en eje x
        [0,  sy, 0],  # escala en eje y
        [0,  0,  1]
    ])

# Aplica una matriz de transformacion a un conjunto de puntos
def aplicar_transformacion(puntos, matriz):
    # convertir puntos a coordenadas homogeneas (añadir fila de unos)
    puntos_h = np.vstack((puntos, np.ones(puntos.shape[1])))
    # multiplicar matriz de transformacion por puntos homogeneos
    resultado_h = matriz @ puntos_h
    # devolver solo coordenadas x e y
    return resultado_h[:2, :]

# Definir vertices de un triangulo en coordenadas 2D
triangulo = np.array([
    [0, 1, 0],  # coordenadas x de los vertices
    [0, 0, 1]   # coordenadas y de los vertices
])

num_frames = 20  # numero total de frames para la animacion
frames = []      # lista para almacenar cada frame generado

# Directorio para guardar imagenes de cada frame
frames_dir = "frames"
if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)

# Configurar figura y ejes para grafico y texto de matriz
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
# Ajustes del eje de la izquierda (grafico de la figura)
ax1.set_xlim(-4, 4)
ax1.set_ylim(-4, 4)
ax1.set_aspect('equal')
ax1.grid(True)
ax1.set_title('Transformacion de la figura')

# Ajustes del eje de la derecha (mostrar matriz de transformacion)
ax2.axis('off')
ax2.set_title('Matriz de transformacion')

# Bucle para generar cada frame de la animacion
for i in range(num_frames):
    print(f"Procesando frame {i+1}/{num_frames}...")
    t = i / (num_frames - 1)  # parametro normalizado de 0 a 1
    
    # calcular parametros de transformacion en funcion de t
    angulo = 360 * t  # rotacion completa de 0 a 360 grados
    escala_factor = 1 + 0.5 * np.sin(2 * np.pi * t)  # escala variable
    tx = 2 * np.sin(2 * np.pi * t)  # traslacion en x
    ty = 2 * np.cos(2 * np.pi * t)  # traslacion en y
    
    # construir matrices de transformacion individuales
    T = traslacion_matriz(tx, ty)
    R = rotacion_matriz(angulo)
    S = escala_matriz(escala_factor, escala_factor)
    
    # combinar transformaciones: primero escalar, luego rotar, luego trasladar
    M = T @ R @ S
    
    # aplicar matriz combinada al triangulo original
    triangulo_transformado = aplicar_transformacion(triangulo, M)
    
    # limpiar ejes antes de dibujar
    ax1.clear()
    ax2.clear()
    
    # reconfigurar limites y estilos de ejes despues de limpiar
    ax1.set_xlim(-4, 4)
    ax1.set_ylim(-4, 4)
    ax1.set_aspect('equal')
    ax1.grid(True)
    ax1.set_title('Transformacion de la figura')
    
    # dibujar triangulo original como guia
    ax1.plot(triangulo[0], triangulo[1], 'b--', alpha=0.3)
    # dibujar triangulo transformado relleno de color rojo
    ax1.fill(triangulo_transformado[0], triangulo_transformado[1], 'r', alpha=0.6)
    
    # preparar texto con matriz de transformacion y parametros
    ax2.axis('off')
    ax2.set_title('Matriz de transformacion')
    matriz_texto = "Matriz de transformacion:\n\n"
    matriz_texto += f"[{M[0,0]:.2f}  {M[0,1]:.2f}  {M[0,2]:.2f}]\n"
    matriz_texto += f"[{M[1,0]:.2f}  {M[1,1]:.2f}  {M[1,2]:.2f}]\n"
    matriz_texto += f"[{M[2,0]:.2f}  {M[2,1]:.2f}  {M[2,2]:.2f}]"
    info_texto = f"\n\nTraslacion: ({tx:.2f}, {ty:.2f})\nRotacion: {angulo:.1f}\nEscala: {escala_factor:.2f}"
    ax2.text(0.5, 0.5, matriz_texto + info_texto, fontsize=12, ha='center', va='center', 
             family='monospace', bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    # guardar imagen del frame actual
    frame_path = f"{frames_dir}/frame_{i:03d}.png"
    plt.tight_layout()
    plt.savefig(frame_path)
    
    # leer imagen guardada y agregarla a la lista de frames
    try:
        frames.append(imageio.imread(frame_path))
    except Exception as e:
        print(f"Error al leer frame {frame_path}: {e}")

# crear GIF animado a partir de los frames
try:
    gif_path = 'transformacion_animada.gif'
    imageio.mimsave(gif_path, frames, duration=100, loop=0)
    print(f"Animacion guardada como '{gif_path}'")
except Exception as e:
    print(f"Error al crear GIF: {e}")

# mostrar la figura final en pantalla
plt.show()
